
# Toxic Comment Detection — Text CNN (Keras, Single CSV, 50k rows)

This notebook trains a **Text CNN** on your single CSV: **`/mnt/data/toxic_comments_50k.csv`**.
We'll do an 80/10/10 split (train/val/test), build a CNN with multi-kernel Conv1D, and evaluate with ROC‑AUC & F1.
Per‑class thresholds are tuned on the validation set for better F1.

> **CSV columns expected:** `id, comment_text, toxic, severe_toxic, obscene, threat, insult, identity_hate`.


## 1) Setup (fast imports; TensorFlow imported later)

In [ ]:

# Keep imports light to avoid slow startup
import os, random, json, math
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, classification_report

# Reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Paths & config
CSV_PATH   = Path("/mnt/data/toxic_comments_50k.csv")
ART_DIR    = Path("./artifacts"); ART_DIR.mkdir(parents=True, exist_ok=True)
MAX_WORDS  = 30000   # vocab size
MAX_LEN    = 200     # tokens per comment
EMBED_DIM  = 100     # use 100 if enabling GloVe
BATCH_SIZE = 64
EPOCHS     = 6

assert CSV_PATH.exists(), f\"File not found: {CSV_PATH}

## 2) Load CSV & quick sanity check

In [ ]:

df = pd.read_csv(CSV_PATH)
print("Shape:", df.shape)
print("Columns:", list(df.columns))
display(df.head())

label_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
for c in label_cols:
    assert c in df.columns, f"Missing label column: {c}"
assert 'comment_text' in df.columns, "Missing 'comment_text' column"

# Ensure labels are 0/1 ints
for c in label_cols:
    df[c] = df[c].astype(int)

# Basic stats
print("\nLabel positives:")
print(df[label_cols].sum())


## 3) Light text cleaning

In [ ]:

import re
URL_RE = re.compile(r'http\S+|www\.\S+')
USER_RE = re.compile(r'@\w+')
HTML_RE = re.compile(r'<.*?>')
SPACE_RE= re.compile(r'\s+')

def clean_text(s: str) -> str:
    if not isinstance(s, str): return ""
    s = s.lower()
    s = URL_RE.sub(' URL ', s)
    s = USER_RE.sub(' USER ', s)
    s = HTML_RE.sub(' ', s)
    s = SPACE_RE.sub(' ', s).strip()
    return s

df['clean_text'] = df['comment_text'].astype(str).apply(clean_text)
display(df[['comment_text','clean_text']].head(3))


## 4) Tokenize & pad sequences (Keras Tokenizer)

In [ ]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token='[OOV]')
tokenizer.fit_on_texts(df['clean_text'].tolist())

def texts_to_padded(texts):
    seqs = tokenizer.texts_to_sequences(texts)
    return pad_sequences(seqs, maxlen=MAX_LEN, padding='post', truncating='post')

X_all = texts_to_padded(df['clean_text'].tolist())
y_all = df[label_cols].values.astype('float32')

print("Tokenized:", X_all.shape, y_all.shape)


## 5) Train/Val/Test split (80/10/10, stratified)

In [ ]:

any_toxic = (y_all.sum(axis=1) > 0).astype(int)

# First split train vs temp
X_train, X_temp, y_train, y_temp, idx_train, idx_temp = train_test_split(
    X_all, y_all, np.arange(len(y_all)), test_size=0.2, random_state=SEED, stratify=any_toxic
)

# Split temp into val/test
any_toxic_temp = (y_temp.sum(axis=1) > 0).astype(int)
X_val, X_test, y_val, y_test, idx_val, idx_test = train_test_split(
    X_temp, y_temp, idx_temp, test_size=0.5, random_state=SEED, stratify=any_toxic_temp
)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


## 6) (Optional) Load GloVe embeddings (skip if not available)

In [ ]:

import numpy as np
from pathlib import Path

GLOVE_PATH = Path("/mnt/data/glove.6B.100d.txt")  # put file here to enable
emb_matrix = None

if GLOVE_PATH.exists():
    print("Loading GloVe from", GLOVE_PATH)
    embeddings_index = {}
    with open(GLOVE_PATH, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.rstrip().split(' ')
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    word_index = tokenizer.word_index
    num_tokens = min(MAX_WORDS, len(word_index) + 1)
    emb_matrix = np.random.normal(0, 0.6, (num_tokens, EMBED_DIM)).astype('float32')
    for word, i in word_index.items():
        if i >= MAX_WORDS: 
            continue
        vec = embeddings_index.get(word)
        if vec is not None:
            emb_matrix[i] = vec
    print("Built embedding matrix:", emb_matrix.shape)
else:
    print("GloVe not found; will train embeddings from scratch.")


## 7) Build Text CNN model (lazy-import TensorFlow)

In [ ]:

# Speed up TF import/startup for CPU-only
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["CUDA_VISIBLE_DEVICES"] = ""     # skip GPU probing if no CUDA
os.environ["TF_NUM_INTEROP_THREADS"] = "2"  # optional: tune threads
os.environ["TF_NUM_INTRAOP_THREADS"]  = "4"

import tensorflow as tf
tf.keras.utils.set_random_seed(SEED)

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Embedding, Conv1D, GlobalMaxPooling1D,
                                     Concatenate, Dense, Dropout)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

def build_text_cnn(max_words, max_len, embed_dim, emb_matrix=None, num_labels=6, filters=128, drop=0.5):
    inp = Input(shape=(max_len,), name="tokens")
    if emb_matrix is not None:
        emb = Embedding(input_dim=emb_matrix.shape[0],
                        output_dim=emb_matrix.shape[1],
                        weights=[emb_matrix],
                        input_length=max_len,
                        trainable=False, name="embedding")(inp)
    else:
        emb = Embedding(input_dim=max_words,
                        output_dim=embed_dim,
                        input_length=max_len,
                        name="embedding")(inp)

    convs = []
    for k in [3,4,5]:
        c = Conv1D(filters, kernel_size=k, activation='relu', padding='valid')(emb)
        p = GlobalMaxPooling1D()(c)
        convs.append(p)
    x = Concatenate()(convs)
    x = Dropout(drop)(x)
    out = Dense(num_labels, activation='sigmoid')(x)

    model = Model(inputs=inp, outputs=out)
    model.compile(optimizer=tf.keras.optimizers.Adam(2e-3),
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.AUC(name='auc', multi_label=True)])
    return model

model = build_text_cnn(MAX_WORDS, MAX_LEN, EMBED_DIM, emb_matrix, num_labels=len(label_cols))
model.summary()


## 8) Train (EarlyStopping on val AUC)

In [ ]:

callbacks = [
    EarlyStopping(monitor='val_auc', mode='max', patience=2, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_auc', mode='max', factor=0.5, patience=1, min_lr=1e-5, verbose=1)
]
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)


## 9) Evaluate on validation & test sets

In [ ]:

def evaluate_probs(y_true, y_prob, label_cols):
    # ROC-AUC per class
    aucs = {}
    for i, lab in enumerate(label_cols):
        try:
            aucs[lab] = roc_auc_score(y_true[:, i], y_prob[:, i])
        except ValueError:
            aucs[lab] = np.nan
    macro_auc = np.nanmean(list(aucs.values()))
    # F1 at 0.5 threshold
    y_pred = (y_prob >= 0.5).astype(int)
    report = classification_report(y_true, y_pred, target_names=label_cols, zero_division=0)
    return aucs, macro_auc, report

# Validation
y_val_prob = model.predict(X_val, batch_size=256, verbose=0)
val_aucs, val_macro_auc, val_report = evaluate_probs(y_val, y_val_prob, label_cols)
print("Validation ROC-AUC per class:", val_aucs)
print(f"Validation Macro ROC-AUC: {val_macro_auc:.4f}")
print("\nValidation classification report (thr=0.5):\n", val_report)

# Test
y_test_prob = model.predict(X_test, batch_size=256, verbose=0)
test_aucs, test_macro_auc, test_report = evaluate_probs(y_test, y_test_prob, label_cols)
print("Test ROC-AUC per class:", test_aucs)
print(f"Test Macro ROC-AUC: {test_macro_auc:.4f}")
print("\nTest classification report (thr=0.5):\n", test_report)


## 10) Tune per-class thresholds (maximize F1 on validation)

In [ ]:

def optimal_thresholds(probs, targets, steps=200):
    thrs = []
    for i in range(probs.shape[1]):
        best_f1, best_t = 0.0, 0.5
        for t in np.linspace(0.05, 0.95, steps):
            f1 = f1_score(targets[:, i], (probs[:, i] >= t).astype(int), zero_division=0)
            if f1 > best_f1:
                best_f1, best_t = f1, t
        thrs.append(best_t)
    return np.array(thrs)

thr_vec = optimal_thresholds(y_val_prob, y_val)
print("Per-class optimal thresholds:")
print(dict(zip(label_cols, np.round(thr_vec, 3))))

# Re-evaluate on test with tuned thresholds
y_test_pred_tuned = (y_test_prob >= thr_vec).astype(int)
print("\nTest report with tuned thresholds:")
print(classification_report(y_test, y_test_pred_tuned, target_names=label_cols, zero_division=0))


## 11) Save artifacts (model, tokenizer, thresholds)

In [ ]:

# Use Keras v3 format to avoid HDF5 dependency
model_path = ART_DIR / "text_cnn_toxic.keras"
tok_path   = ART_DIR / "tokenizer.json"
thr_path   = ART_DIR / "thresholds.npy"

with open(tok_path, "w") as f:
    f.write(tokenizer.to_json())
np.save(thr_path, thr_vec)

model.save(model_path)

print("Saved model to:", model_path.resolve())
print("Saved tokenizer to:", tok_path.resolve())
print("Saved thresholds to:", thr_path.resolve())


## 12) Inference helper (single text or list)

In [ ]:

def predict_texts(texts, model, tokenizer, max_len, thresholds=None):
    if isinstance(texts, str):
        texts = [texts]
    # reuse cleaner
    cleaned = [clean_text(t) for t in texts]
    seqs = tokenizer.texts_to_sequences(cleaned)
    X = pad_sequences(seqs, maxlen=max_len, padding='post', truncating='post')
    probs = model.predict(X, verbose=0)
    if thresholds is None:
        preds = (probs >= 0.5).astype(int)
    else:
        preds = (probs >= thresholds).astype(int)
    return probs, preds

sample_texts = [
    "Thanks for the detailed explanation, much appreciated!",
    "You're clueless and incompetent, just stop talking.",
    "Keep talking and you'll regret it."
]
probs, preds = predict_texts(sample_texts, model, tokenizer, MAX_LEN, thresholds=thr_vec)
print("Sample predictions (probabilities):\n", probs.round(3))
print("Sample predictions (labels):\n", preds)


## 13) (Optional) Baseline: TF‑IDF + Logistic Regression (fast sanity check)

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Small baseline on train/val split only (to keep it quick)
tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=100000, min_df=2)
X_tr_tfidf = tfidf.fit_transform(df.loc[idx_train, 'clean_text'].tolist())
X_va_tfidf = tfidf.transform(df.loc[idx_val, 'clean_text'].tolist())

lr_reports = {}
for i, lab in enumerate(label_cols):
    lr = LogisticRegression(max_iter=200, n_jobs=None)
    lr.fit(X_tr_tfidf, y_train[:, i])
    val_prob = lr.predict_proba(X_va_tfidf)[:, 1]
    auc = roc_auc_score(y_val[:, i], val_prob)
    pred = (val_prob >= 0.5).astype(int)
    f1 = f1_score(y_val[:, i], pred, zero_division=0)
    lr_reports[lab] = {"auc": round(auc, 4), "f1@0.5": round(f1, 4)}
lr_reports
